In [1]:
import boto3
import xesmf as xe
import xarray as xr
import netCDF4
import pandas as pd
import argparse
import base64
import boto3
import json
import requests
import os
import platform
from subprocess import Popen
import shutil
from datetime import datetime
import re
import pandas as pd
from load_credentials import *
from bbox import * 
from netCDF4 import Dataset
import netCDF4 as nc
from bbox import * 
import numpy as np
from threading import Thread
import pandas as pd
import numpy as np
import boto3
import netCDF4 as nc
import xarray as xr
import xesmf as xe
from dask import delayed, compute
from dask.distributed import Client, LocalCluster

# Define event with credentials and S3 endpoint details
event = {
    's3_endpoint': 'https://data.asdc.earthdata.nasa.gov/s3credentials',  # replace with actual endpoint
    'edl_username': 'amanda.murray19',  # replace with your EDL username
    'edl_password': 'Sat_modeling_berk2024',  # replace with your EDL password
    'bucket_name': 'asdc-prod-protected/TEMPO/TEMPO_NO2_L3_V03'  # replace with your bucket name
}

netrc_path = os.path.expanduser('~/.netrc')  # Expands to the user's home directory
if os.path.exists(netrc_path):
    print(".netrc file exists.")
    os.remove(netrc_path)
    print(".netrc file has been removed.")
else:
    print(".netrc file does not exist.")# Retrieve credentials

.netrc file does not exist.


In [2]:
df = pd.read_csv("../../data/tempo_data/no2_tempo_files_df.csv")
df['time_hr_ct'] = pd.to_datetime(df['timestamp_ct']).dt.floor('h')

result_df = df.groupby(['date_central', 'Hour'], as_index=False).first()
result_df['time_hr_ct'] = pd.to_datetime(result_df['timestamp_ct']).dt.floor('h')
result_df[['time_hr_ct']].to_csv('../../data/tempo_data/no2_file_hours.csv')

df['month_year'] = pd.to_datetime(df['date_central']).dt.to_period('M')
df['year'] = pd.to_datetime(df['date_central']).dt.year
date_groups = df.groupby('month_year')

In [4]:
df_2025 = df[df['year']==2025]
date_groups = df_2025.groupby('month_year')

# Set up a Dask distributed cluster
cluster = LocalCluster()
client = Client(cluster)

# Function to process a single month/year group
@delayed
def process_group(group_name, group_df):
    local_null_data = []
    date_list = group_df['date_central'].unique()

    for date in date_list:
        creds = retrieve_credentials(event)
        
        client = boto3.client(
            's3',
            aws_access_key_id=creds["accessKeyId"],
            aws_secret_access_key=creds["secretAccessKey"],
            aws_session_token=creds["sessionToken"]
        )
        file_df = group_df[group_df['date_central'] == date]
        filepaths = list(file_df['FilePath'])
        full_time = list(file_df['timestamp_ct'])
        daily_xr_list = []
        
        for i in range(len(filepaths)):
            file = filepaths[i]
            new_time = str(np.array(full_time[i]))

            bucket_name = 'asdc-prod-protected'
            object_key = file
            local_file_name = f'current_sat_{group_name}.nc'
            client.download_file(bucket_name, object_key, local_file_name)

            # Processing the NetCDF file
            ds = nc.Dataset(local_file_name, mode='r')
            support_data_xr = ds.groups['support_data']
            support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
            support_data_xr = support_data_xr[['vertical_column_total', 'eff_cloud_fraction']]
            tempo = xr.open_dataset(local_file_name, group="/product")
            tempo = xr.merge([tempo, support_data_xr])
            lat_lon_coords = xr.open_dataset(local_file_name)
            lat = lat_lon_coords.coords['latitude'].values
            lon = lat_lon_coords.coords['longitude'].values
            tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])

            # Filtering and regridding
            louisiana_data = tempo.sel(
                latitude=(tempo.latitude >= lat_min) & (tempo.latitude <= lat_max),
                longitude=(tempo.longitude >= lon_min) & (tempo.longitude <= lon_max)
            )
            louisiana_data = louisiana_data.where(~(louisiana_data['main_data_quality_flag'] > 1), np.nan)
            # lat_new = np.arange(lat_min + 0.005, lat_max, 0.01)
            # lon_new = np.arange(lon_min + 0.005, lon_max, 0.01)
            # ds_out = xr.Dataset({"lat": (["lat"], lat_new), "lon": (["lon"], lon_new)})
            # regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")
            # louisiana_data_regrid = regridder(louisiana_data)
            daily_xr_list.append(louisiana_data)

            # Null count and percentage
            null_count = int(louisiana_data.vertical_column_total.isnull().sum().values)
            total_elements = louisiana_data.vertical_column_total.size
            nan_percentage = round((null_count / total_elements) * 100, 0)
            new_row = {'date': date, 'file': file, 'time': new_time, 'real_time': full_time, 
                       'null_count': null_count, 'percent_null': nan_percentage}
            local_null_data.append(new_row)

        combined_data = xr.concat(daily_xr_list, dim='time')
        combined_data.to_netcdf(f'../../data/tempo_data/no2_daily_files/tempo_{date}.nc')

    # Save satellite_null_data for the month/year
    local_null_df = pd.DataFrame(local_null_data)
    local_null_df.to_csv(f"../../data/tempo_data/satellite_null_data_raw_{group_name}.csv", index=False)
    return f"Completed processing for {group_name}"

# Retrieve credentials
# creds = retrieve_credentials(event)

# Submit tasks to Dask
tasks = [process_group(group_name, group_df) for group_name, group_df in date_groups]

# Execute tasks
results = compute(*tasks)
print(results)


2025-06-29 15:59:31,400 - distributed.scheduler - WARNING - Worker failed to heartbeat for 494s; attempting restart: <WorkerState 'tcp://127.0.0.1:40045', name: 3, status: running, memory: 0, processing: 2>
2025-06-29 15:59:35,405 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-06-29 15:59:36,405 - distributed.core - ERROR - Exception while handling op kill
Traceback (most recent call last):
  File "/opt/conda/envs/ds_env/lib/python3.11/site-packages/distributed/utils.py", line 1910, in wait_for
    return await fut
           ^^^^^^^^^
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/envs/ds_env/lib/python3.11/site-packages/distributed/core.py", line 834, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "/opt/conda/envs/ds_env/lib/python3.11/site-packages/distributed/nanny.py", line 400, in kill
    await sel

('Completed processing for 2025-01', 'Completed processing for 2025-02', 'Completed processing for 2025-03', 'Completed processing for 2025-04', 'Completed processing for 2025-05', 'Completed processing for 2025-06')


In [3]:
### WHEN RERUN, SPLIT UP INTO GROUPS 
df_2024 = df[df['year']==2024]
df_2024 = df_2024[(df_2024['month_year'].astype(str)=='2024-11')|(df_2024['month_year'].astype(str)=='2024-12')]
# df_2024 = df_2024[df_2024['month_year'].astype(str)!='2024-01']
# df_2024 = df_2024[df_2024['month_year'].astype(str)!='2024-04']
# df_2024 = df_2024[df_2024['month_year'].astype(str)!='2024-05']
# df_2024 = df_2024[df_2024['month_year'].astype(str)!='2024-11']

date_groups = df_2024.groupby('month_year')

In [4]:
df_2024

,Unnamed: 0,FilePath,Date,FileNumber,L2_File,BeginningDateTime,EndingDateTime,Time,timestamp_ct,date_central,Hour,Minute,time_hr_ct,month_year,year
6225,6225,TEMPO/TEMPO_NO2_L3_V03/2024.11.01/TEMPO_NO2_L3...,20241101,S002,3,2024-11-01 12:41:41+00:00,2024-11-01 13:01:32+00:00,2024-11-01 12:51:36.500000+00:00,2024-11-01 07:51:36,2024-11-01,7,51,2024-11-01 07:00:00,2024-11,2024
6226,6226,TEMPO/TEMPO_NO2_L3_V03/2024.11.01/TEMPO_NO2_L3...,20241101,S003,3,2024-11-01 13:21:46+00:00,2024-11-01 13:41:37+00:00,2024-11-01 13:31:41.500000+00:00,2024-11-01 08:31:41,2024-11-01,8,31,2024-11-01 08:00:00,2024-11,2024
6227,6227,TEMPO/TEMPO_NO2_L3_V03/2024.11.01/TEMPO_NO2_L3...,20241101,S004,3,2024-11-01 14:01:51+00:00,2024-11-01 14:21:42+00:00,2024-11-01 14:11:46.500000+00:00,2024-11-01 09:11:46,2024-11-01,9,11,2024-11-01 09:00:00,2024-11,2024
6228,6228,TEMPO/TEMPO_NO2_L3_V03/2024.11.01/TEMPO_NO2_L3...,20241101,S005,3,2024-11-01 14:41:59+00:00,2024-11-01 15:01:50+00:00,2024-11-01 14:51:54.500000+00:00,2024-11-01 09:51:54,2024-11-01,9,51,2024-11-01 09:00:00,2024-11,2024
6229,6229,TEMPO/TEMPO_NO2_L3_V03/2024.11.01/TEMPO_NO2_L3...,20241101,S006,3,2024-11-01 15:41:59+00:00,2024-11-01 16:01:50+00:00,2024-11-01 15:51:54.500000+00:00,2024-11-01 10:51:54,2024-11-01,10,51,2024-11-01 10:00:00,2024-11,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6988,6988,TEMPO/TEMPO_NO2_L3_V03/2024.12.31/TEMPO_NO2_L3...,20241231,S008,3,2024-12-31 18:07:55+00:00,2024-12-31 18:27:46+00:00,2024-12-31 18:17:50.500000+00:00,2024-12-31 12:17:50,2024-12-31,12,17,2024-12-31 12:00:00,2024-12,2024
6989,6989,TEMPO/TEMPO_NO2_L3_V03/2024.12.31/TEMPO_NO2_L3...,20241231,S009,3,2024-12-31 19:07:55+00:00,2024-12-31 19:27:46+00:00,2024-12-31 19:17:50.500000+00:00,2024-12-31 13:17:50,2024-12-31,13,17,2024-12-31 13:00:00,2024-12,2024
6990,6990,TEMPO/TEMPO_NO2_L3_V03/2024.12.31/TEMPO_NO2_L3...,20241231,S010,3,2024-12-31 20:07:55+00:00,2024-12-31 20:27:46+00:00,2024-12-31 20:17:50.500000+00:00,2024-12-31 14:17:50,2024-12-31,14,17,2024-12-31 14:00:00,2024-12,2024
6991,6991,TEMPO/TEMPO_NO2_L3_V03/2024.12.31/TEMPO_NO2_L3...,20241231,S011,3,2024-12-31 21:07:55+00:00,2024-12-31 21:27:46+00:00,2024-12-31 21:17:50.500000+00:00,2024-12-31 15:17:50,2024-12-31,15,17,2024-12-31 15:00:00,2024-12,2024


In [5]:
# df_2024 = df[df['year']==2024]
# date_groups = df_2024.groupby('month_year')

# Set up a Dask distributed cluster
cluster = LocalCluster()
client = Client(cluster)

# Function to process a single month/year group
@delayed
def process_group(group_name, group_df):
    local_null_data = []
    date_list = group_df['date_central'].unique()

    for date in date_list:
        creds = retrieve_credentials(event)
        
        client = boto3.client(
            's3',
            aws_access_key_id=creds["accessKeyId"],
            aws_secret_access_key=creds["secretAccessKey"],
            aws_session_token=creds["sessionToken"]
        )
        file_df = group_df[group_df['date_central'] == date]
        filepaths = list(file_df['FilePath'])
        full_time = list(file_df['timestamp_ct'])
        daily_xr_list = []
        
        for i in range(len(filepaths)):
            file = filepaths[i]
            new_time = str(np.array(full_time[i]))

            bucket_name = 'asdc-prod-protected'
            object_key = file
            local_file_name = f'current_sat_{group_name}.nc'
            client.download_file(bucket_name, object_key, local_file_name)

            # Processing the NetCDF file
            ds = nc.Dataset(local_file_name, mode='r')
            support_data_xr = ds.groups['support_data']
            support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
            support_data_xr = support_data_xr[['vertical_column_total', 'eff_cloud_fraction']]
            tempo = xr.open_dataset(local_file_name, group="/product")
            tempo = xr.merge([tempo, support_data_xr])
            lat_lon_coords = xr.open_dataset(local_file_name)
            lat = lat_lon_coords.coords['latitude'].values
            lon = lat_lon_coords.coords['longitude'].values
            tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])

            # Filtering and regridding
            louisiana_data = tempo.sel(
                latitude=(tempo.latitude >= lat_min) & (tempo.latitude <= lat_max),
                longitude=(tempo.longitude >= lon_min) & (tempo.longitude <= lon_max)
            )
            louisiana_data = louisiana_data.where(~(louisiana_data['main_data_quality_flag'] > 1), np.nan)
            # lat_new = np.arange(lat_min + 0.005, lat_max, 0.01)
            # lon_new = np.arange(lon_min + 0.005, lon_max, 0.01)
            # ds_out = xr.Dataset({"lat": (["lat"], lat_new), "lon": (["lon"], lon_new)})
            # regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")
            # louisiana_data_regrid = regridder(louisiana_data)
            daily_xr_list.append(louisiana_data)

            # Null count and percentage
            null_count = int(louisiana_data.vertical_column_total.isnull().sum().values)
            total_elements = louisiana_data.vertical_column_total.size
            nan_percentage = round((null_count / total_elements) * 100, 0)
            new_row = {'date': date, 'file': file, 'time': new_time, 'real_time': full_time, 
                       'null_count': null_count, 'percent_null': nan_percentage}
            local_null_data.append(new_row)

        combined_data = xr.concat(daily_xr_list, dim='time')
        combined_data.to_netcdf(f'../../data/tempo_data/no2_daily_files/tempo_{date}.nc')

    # Save satellite_null_data for the month/year
    local_null_df = pd.DataFrame(local_null_data)
    local_null_df.to_csv(f"../../data/tempo_data/satellite_null_data_raw_{group_name}.csv", index=False)
    return f"Completed processing for {group_name}"

# Retrieve credentials
# creds = retrieve_credentials(event)

# Submit tasks to Dask
tasks = [process_group(group_name, group_df) for group_name, group_df in date_groups]

# Execute tasks
results = compute(*tasks)
print(results)


('Completed processing for 2024-11', 'Completed processing for 2024-12')


In [ ]:
# # Function to process a single month/year group
# def process_group(group_name, group_df, creds):
#     local_null_data = []
#     date_list = group_df['date_central'].unique()

#     for date in date_list:
#         client = boto3.client(
#             's3',
#             aws_access_key_id=creds["accessKeyId"],
#             aws_secret_access_key=creds["secretAccessKey"],
#             aws_session_token=creds["sessionToken"]
#         )
#         file_df = group_df[group_df['date_central'] == date]
#         filepaths = list(file_df['FilePath'])
#         full_time = list(file_df['timestamp_ct'])
#         daily_xr_list = []

#         for i in range(len(filepaths)):
#             file = filepaths[i]
#             new_time = str(np.array(full_time[i]))

#             bucket_name = 'asdc-prod-protected'
#             object_key = file
#             local_file_name = f'current_sat_{group_name}.nc'
#             client.download_file(bucket_name, object_key, local_file_name)

#             # Processing the NetCDF file
#             ds = nc.Dataset(local_file_name, mode='r')
#             support_data_xr = ds.groups['support_data']
#             support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
#             support_data_xr = support_data_xr[['vertical_column_total', 'eff_cloud_fraction', 'surface_pressure']]
#             tempo = xr.open_dataset(local_file_name, group="/product")
#             tempo = xr.merge([tempo, support_data_xr])
#             lat_lon_coords = xr.open_dataset(local_file_name)
#             lat = lat_lon_coords.coords['latitude'].values
#             lon = lat_lon_coords.coords['longitude'].values
#             tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])

#             # Filtering and regridding
#             louisiana_data = tempo.sel(
#                 latitude=(tempo.latitude >= lat_min) & (tempo.latitude <= lat_max),
#                 longitude=(tempo.longitude >= lon_min) & (tempo.longitude <= lon_max)
#             )
#             louisiana_data = louisiana_data.where(~(louisiana_data['main_data_quality_flag'] > 1), np.nan)
           
#             lat_new = np.arange(lat_min + 0.005, lat_max, 0.01)
#             lon_new = np.arange(lon_min + 0.005, lon_max, 0.01)
#             ds_out = xr.Dataset({"lat": (["lat"], lat_new), "lon": (["lon"], lon_new)})
#             if louisiana_data.dims:
#                 regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")
#                 louisiana_data_regrid = regridder(louisiana_data)
#                 daily_xr_list.append(louisiana_data_regrid)

#                 # Null count and percentage
#                 null_count = int(louisiana_data_regrid.vertical_column_total.isnull().sum().values)
#                 total_elements = louisiana_data_regrid.vertical_column_total.size
#                 nan_percentage = round((null_count / total_elements) * 100, 0)
#                 new_row = {'date': date, 'file': file, 'time': new_time, 'real_time': full_time, 
#                            'null_count': null_count, 'percent_null': nan_percentage}
#                 local_null_data.append(new_row)
            
#             else:
#                 print(f"Skipping regridding for date: {date} as data is empty!")
#                 continue

#         combined_data = xr.concat(daily_xr_list, dim='time')
#         combined_data.to_netcdf(f'../../data/tempo_data/no2_daily_files/tempo_{date}.nc')

#     # Save satellite_null_data for the month/year
#     local_null_df = pd.DataFrame(local_null_data)
#     local_null_df.to_csv(f"../../data/tempo_data/satellite_null_data_{group_name}.csv", index=False)

# # Main thread management
# def process_all_groups():
#     creds = retrieve_credentials(event)  # Retrieve credentials once
#     threads = []

#     for group_name, group_df in date_groups:
#         thread = Thread(target=process_group, args=(group_name, group_df, creds))
#         threads.append(thread)
#         thread.start()

#     # Wait for all threads to complete
#     for thread in threads:
#         thread.join()

# process_all_groups()

In [ ]:
# ### TIME 5 days = 875 s (~15 min)
# # Initialize the Google Cloud Storage client
# # Example DataFrame (assuming it exists)
# satellite_null_data = pd.DataFrame({
#     'date': [],
#     'file': [],
#     'time': [],
#     'null_count': [],
#     'percent_null': []
# })

# date_list = list(df['date_central'].unique())

# for date in date_list:
#     # Retrieve credentials
#     creds = retrieve_credentials(event)

#     # Use the credentials to access the S3 bucket
#     client = boto3.client('s3',
#         aws_access_key_id=creds["accessKeyId"],
#         aws_secret_access_key=creds["secretAccessKey"],
#         aws_session_token=creds["sessionToken"]
#     )
    
#     file_df = df[df['date_central']==date]
#     filepaths = list(file_df['FilePath'])
#     # times = list(file_df['time_hr_ct'])
#     full_time = list(file_df['timestamp_ct'])
    
#     daily_xr_list = []
    
#     for i in range(len(filepaths)):
#         file=filepaths[i]
#         new_time = str(np.array(full_time[i]))
        
#         bucket_name = 'asdc-prod-protected'
#         object_key = file
#         local_file_name = 'current_sat.nc'
#         client.download_file(bucket_name, object_key, local_file_name)
        
#         ds = nc.Dataset('current_sat.nc', mode='r')
#         support_data_xr = ds.groups['support_data']
#         support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
#         support_data_xr=support_data_xr[['vertical_column_total','eff_cloud_fraction']]
#         tempo = xr.open_dataset('current_sat.nc', group="/product")
#         tempo = xr.merge([tempo, support_data_xr])
#         del ds
#         del support_data_xr
#         lat_lon_coords = xr.open_dataset('current_sat.nc')
#         lat = lat_lon_coords.coords['latitude'].values
#         lon = lat_lon_coords.coords['longitude'].values
#         tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
#         del lat
#         del lon

#         # Filter the data using the .sel method for selecting ranges in xarray
#         louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
#         louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
#         del tempo
        
#         mask = louisiana_data['main_data_quality_flag'] >1 
#         louisiana_data = louisiana_data.where(~mask, other=np.nan)
#         mask = louisiana_data['main_data_quality_flag'].isnull()
#         louisiana_data = louisiana_data.where(~mask, other=np.nan)
        
#         lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
#         lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

#         ds_out = xr.Dataset(
#             {
#                 "lat": (["lat"], lat_new),
#                 "lon": (["lon"], lon_new),
#             }
#         )

#         # Create the regridder object.
#         regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")

#         # Apply the regridding operation.
#         louisiana_data_regrid = regridder(louisiana_data)
#         daily_xr_list.append(louisiana_data_regrid)
        
#         null_count = int(louisiana_data_regrid.vertical_column_troposphere.isnull().sum().values)
#         total_elements = louisiana_data_regrid.vertical_column_troposphere.size
#         nan_percentage = round((null_count / total_elements) * 100,0)
        
#         new_row = {'date':date, 'file':file, 'time':new_time, 'real_time':full_time, 'null_count':null_count, 'percent_null':nan_percentage}
#         # Convert the new row to a DataFrame
#         new_row_df = pd.DataFrame([new_row])
#         # Append the new row to the DataFrame using concat
#         satellite_null_data = pd.concat([satellite_null_data, new_row_df], ignore_index=True)
        
#     combined_data = xr.concat(daily_xr_list, dim='time')
    
#     combined_data.to_netcdf(f'../../data/tempo_data/daily_files/tempo_{date}.nc')
         

# satellite_null_data.to_csv("../../data/tempo_data/satellite_null_data_09_2023.csv")

In [ ]:
# import matplotlib.pyplot as plt
# # Open the file with xarray
# ds = xr.open_dataset('daily.nc')
# # Assuming your xarray is called 'x'
# # Define the bounding box for Louisiana
# lat_min, lat_max = 28.92, 33.02
# lon_min, lon_max = -94.04, -88.82

# subset = ds.sel(time='2023-08-07 07:00:00')

# # Plot the weight variable in Louisiana's bounding box
# plt.figure(figsize=(8, 6))
# subset.vertical_column_troposphere.plot(cmap='viridis')
# plt.title('Weight Variable in Louisiana Bounding Box')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.grid(True)
# plt.show()